# Exercise: SINDy

Exercise on the implementation of SYNDy

Author: Stefano Pagani <stefano.pagani@polimi.it>.

Date: 2024

Course: Mathematical and numerical foundations of scientific machine learning.

Example adapted from:
[1] Brunton, S. L., & Kutz, J. N. (122).
Data-driven science and engineering: Machine learning,
dynamical systems, and control. Cambridge University Press.

In [ ]:
# imports

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from mpl_toolkits.mplot3d import Axes3D
from scipy import integrate


In [ ]:

# figure setting
rcParams.update({'font.size': 18})
plt.rcParams['figure.figsize'] = [12, 12]


Numerical simulation of the Lorenz System

In [ ]:

dt = 0.01
T = 50
t = np.arange(dt,T+dt,dt)
t_p1 = np.arange(dt,T+2*dt,dt)
beta = 8/3
sigma = 10
rho = 28
n = 3

def lorenz_deriv(x_y_z, t0, sigma=sigma, beta=beta, rho=rho):
    x, y, z = x_y_z
    return [sigma * (y - x), x * (rho - z) - y, x * y - beta * z]

np.random.seed(123)
# intial condition
x0 = (-8,8,27)

# ODE solver
x = integrate.odeint(lorenz_deriv, x0, t,rtol=10**(-12),atol=10**(-12)*np.ones_like(x0))

x_p1 = integrate.odeint(lorenz_deriv, x0, t_p1,rtol=10**(-12),atol=10**(-12)*np.ones_like(x0))
# noisy measurement
# add noise


In [ ]:

# plot

fig,ax = plt.subplots(1,1,subplot_kw={'projection': '3d'})

ax.plot(x[:,0], x[:,1], x[:,2],linewidth=1)

ax.view_init(18, -113)
plt.show()


In [ ]:

## Compute Derivative
dx = np.zeros_like(x)
for j in range(len(t)):
    # exact
    dx[j,:] = lorenz_deriv(x[j,:],0,sigma,beta,rho)
    # numerically approximated 
    #dx[j,:] = 
    

Task 1: complete the dictonary of candidate functions

In [ ]:

## SINDy Function Definitions

def poolData(yin,nVars,polyorder):
    n = yin.shape[0]
    yout = np.zeros((n,1))

    # poly order 0
    yout[:,0] = np.ones(n)

    # poly order 1
    for i in range(nVars):
        yout = np.append(yout,yin[:,i].reshape((yin.shape[0],1)),axis=1)

    # poly order 2
    #if polyorder >= 2:
        #for i in range(nVars):
            # ...

    # poly order 3
    # ...

    return yout

def SINDy(Theta,dXdt,lamb,n,maxiter):
    Xi = np.linalg.lstsq(Theta,dXdt,rcond=None)[0] # Initial guess: Least-squares

    for k in range(maxiter):
        smallinds = np.abs(Xi) < lamb # Find small coefficients
        Xi[smallinds] = 0                          # and threshold
        for ind in range(n):                       # n is state dimension
            biginds = smallinds[:,ind] == 0
            # Regress dynamics onto remaining terms to find sparse Xi
            Xi[biginds,ind] = np.linalg.lstsq(Theta[:,biginds],dXdt[:,ind],rcond=None)[0]

    return Xi


In [ ]:

Theta = poolData(x,n,3) # Up to third order polynomials
lamb = 0.0025 # sparsification knob lambda
Xi = SINDy(Theta,dx,lamb,n,10)

# vector of coefficients
print(Xi)

Task 2: substitute exact derivative with a numerical approximation of the derivative

Task 3: add noise to the measurement to test the robustness of the reconstruction